In [1]:
import pandas as pd

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

import os

import pdir

In [2]:
# Script inputs
    
FILE_PATH = "../orbsim/r4b_3d/ephemerides/"

PLANETS = ["earth",
           "mars"]

END_YEAR = "2039"

INTERPOLATION_POINTS = 23  # number of interpolated points on line segnment between two table position values.
                          # e.g. 9 will give 0.1, 0.2, ..., 0.8, 0.9 on the segment between 0 and 1.

In [3]:
ephemerides_filename_dict = {}

for planet in PLANETS:
    ephemerides_filename_dict[planet] = "{}_2019-{}.csv".format(planet, END_YEAR)

ephemerides_filename_dict

{'earth': 'earth_2019-2039.csv', 'mars': 'mars_2019-2039.csv'}

In [4]:
# Change CWD of necessary
cwd = os.getcwd()
in_correct_cwd = 'code' + FILE_PATH[2:-1] == cwd[-30:] # Check if last part of cwd is '/code/orbsim/r4b_3d'

if not in_correct_cwd:    
    os.chdir(FILE_PATH)
    cwd = os.getcwd()

print(cwd)

/Users/gandalf/Dropbox/repositories/letomes/code/orbsim/r4b_3d/ephemerides


In [5]:
# Read CSV files into dict
ephemerides = {}
for body, csv_filename in ephemerides_filename_dict.items():
    ephemerides[body] = pd.read_csv(csv_filename, parse_dates=['date'])

pd.set_option("max_row", 5)
ephemerides    

{'earth':          day      MJD       date         r      theta         phi         x  \
 0        0.0  58484.0 2019-01-01  0.983311  66.939900  100.873526 -0.170672   
 1        1.0  58485.0 2019-01-02  0.983304  67.020209  101.977771 -0.187873   
 ...      ...      ...        ...       ...        ...         ...       ...   
 7304  7304.0  65788.0 2038-12-31  0.983386  66.860323   99.623273 -0.151167   
 7305  7305.0  65789.0 2039-01-01  0.983361  66.931919  100.728669 -0.168423   
 
              y         z  
 0     0.888496  0.385160  
 1     0.885562  0.383888  
 ...        ...       ...  
 7304  0.891548  0.386445  
 7305  0.888917  0.385305  
 
 [7306 rows x 9 columns],
 'mars':          day      MJD       date         r       theta         phi         x  \
 0        0.0  58484.0 2019-01-01  1.453495   74.996659   39.051247  1.090280   
 1        1.0  58485.0 2019-01-02  1.454684   74.803063   39.609720  1.081506   
 ...      ...      ...        ...       ...         ...       

In [6]:
# Linear interpolation of all planet positions

interpolated_ephemerides = {}

for body, eph in ephemerides.items():
    cols = eph.columns
    interpolated_body = pd.DataFrame(columns=cols)
    interpolated_body = interpolated_body.append(eph.iloc[[0]])  # Add initial point

    N = len(eph) - 1  # number of intervals that needs interpolation (0->1, 1->2, ...,)

    for n in range(N):  # add all points for day ]n, n+1]

        start_position_df = eph.iloc[[n]]
        end_position_df = eph.iloc[[n+1]]

        start_position_series = start_position_df.iloc[0]
        end_position_series = end_position_df.iloc[0]

        position_difference_series = end_position_series - start_position_series

        start_day = int(start_position_df['day'])

        for i in range(1,INTERPOLATION_POINTS+1):
            interpolated_position_series = start_position_series + i / (INTERPOLATION_POINTS + 1) * position_difference_series
            interpolated_position_df = pd.DataFrame(data=[interpolated_position_series.values], columns=cols)
            interpolated_body = interpolated_body.append(interpolated_position_df)

        interpolated_body = interpolated_body.append(end_position_df) # Add upper bound in current interval

    interpolated_body.set_index('day', inplace=True)

    interpolated_ephemerides[body] = interpolated_body

In [7]:
interpolated_ephemerides

{'earth':                       MJD                date         r      theta  \
 day                                                                  
 0.000000     58484.000000 2019-01-01 00:00:00  0.983311  66.939900   
 0.041667     58484.041667 2019-01-01 01:00:00  0.983311  66.943247   
 ...                   ...                 ...       ...        ...   
 7304.958333  65788.958333 2038-12-31 23:00:00  0.983362  66.928936   
 7305.000000  65789.000000 2039-01-01 00:00:00  0.983361  66.931919   
 
                     phi         x         y         z  
 day                                                    
 0.000000     100.873526 -0.170672  0.888496  0.385160  
 0.041667     100.919536 -0.171388  0.888373  0.385107  
 ...                 ...       ...       ...       ...  
 7304.958333  100.682611 -0.167704  0.889026  0.385353  
 7305.000000  100.728669 -0.168423  0.888917  0.385305  
 
 [175321 rows x 8 columns],
 'mars':                       MJD                date         

In [8]:
# Write dicts to CSV files

for body, filename in ephemerides_filename_dict.items():
    basename, extension = filename.split('.')
    extended_filename = "{}_interpolated-{}.csv".format(basename,INTERPOLATION_POINTS)
    interpolated_ephemerides[body].to_csv(extended_filename)